In [1]:
import matplotlib
%matplotlib inline

import numpy as np
import time
import cv2

import os
from lava.magma.core.run_configs import Loihi2SimCfg, Loihi2HwCfg
from lava.magma.core.run_conditions import RunSteps, RunContinuous
from lava.magma.core.decorator import implements, requires, tag
from lava.magma.core.model.py.model import PyLoihiProcessModel
from lava.magma.core.model.py.ports import PyOutPort, PyInPort
from lava.magma.core.model.py.type import LavaPyType
from lava.magma.core.process.ports.ports import OutPort, InPort
from lava.magma.core.process.process import AbstractProcess
from lava.magma.core.process.variable import Var
from lava.magma.core.resources import CPU
from lava.magma.core.sync.protocols.loihi_protocol import LoihiProtocol
from lava.proc.io.sink import RingBuffer

# from lava.lib.peripherals.dvs.inivation import InivationCamera
from lava.lib.peripherals.dvs.prophesee import PropheseeCamera
from metavision_core.utils import get_sample
from metavision_sdk_cv import TrailFilterAlgorithm, ActivityNoiseFilterAlgorithm
from metavision_sdk_core import PolarityFilterAlgorithm

from multiprocessing import Pipe
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import IPython


from metavision_core.event_io import RawReader
from lava.lib.peripherals.dvs.transform import Compose, MirrorHorizontally, MirrorVertically, MergePolarities, Downsample
from metavision_ml.preprocessing import histo, histo_quantized
from metavision_core.event_io import RawReader

from lava.proc.embedded_io.spike import PyToNxAdapter, NxToPyAdapter
from lava.proc.lif.process import LIF, LogConfig
from lava.proc.dense.process import Dense
from lava.proc.sparse.process import Sparse
import logging
from lava.proc.dense.models import PyDenseModelBitAcc
from lava.proc.lif.models import PyLifModelBitAcc
from lava.proc.sparse.models import PySparseModelBitAcc
from lava.proc.embedded_io.spike import PyToNxAdapter, NxToPyAdapter

from utils import VisSwipeProcess
from scipy.sparse import csr_matrix
from lava.magma.core.process.ports.connection_config import ConnectionConfig, SpikeIOInterface, SpikeIOPort, SpikeIOMode
from lava.magma.compiler.subcompilers.nc.ncproc_compiler import CompilerOptions
CompilerOptions.verbose = True

/usr/lib/python3/dist-packages/metavision_core_ml/preprocessing/viz.py:41: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def _viz_events(events, img):


In [2]:
SEQUENCE_FILENAME_RAW = "traffic_monitoring.raw"
# if the file doesn't exist, it will be downloaded from Prophesee's public sample server
get_sample(SEQUENCE_FILENAME_RAW)

reader = RawReader(SEQUENCE_FILENAME_RAW)
height, width = reader.get_size()
del reader
# width, height = (640, 480)


In [3]:
connection_config: ConnectionConfig = ConnectionConfig()
connection_config.interface = SpikeIOInterface.PIO
connection_config.spike_io_port = SpikeIOPort.PIO_NORTH
connection_config.spike_io_mode = SpikeIOMode.TIME_COMPARE


## Swipe detector

In the next section, we will create a simple hand gesture recognition model. For this purpose, we create a simple network in Lava in which we hand crafted the connectivity to detect general motion in the input stream.

### Connectivity

There are four population of LIF neurons detecting the directions 'up', 'down', 'left' and 'right'. Each population is recurrently connected with a shifted local kernel. For example the neurons in the up channel connects such that a neuron excites its neurons above. If the movement in the image is indeed upwards, there will be more events in the 'up' population compared to the others. This number of events in each population is used to predict the overall movement in the image.



<table>
<tr>
    <td> UP</td>
    <td> RIGHT </td>
    <td> DOWN </td>
    <td> LEFT </td>
</tr>
<tr>
    <td> <img src="imgs/swipe_kernel_up.png" alt="Drawing" style="height: 250px;"/> </td>
    <td> <img src="imgs/swipe_kernel_right.png" alt="Drawing" style="height: 250px;"/> </td>
    <td> <img src="imgs/swipe_kernel_down.png" alt="Drawing" style="height: 250px;"/> </td>
    <td> <img src="imgs/swipe_kernel_left.png" alt="Drawing" style="height: 250px;"/> </td>
</tr>
</table>

In [4]:

from scipy.sparse import csr_matrix

transformations = Compose(
    [
        Downsample(factor=0.05),
        MergePolarities(),
    ]
)


print("Create Cam")
# Init Processes
# frame_input = InivationCamera(device="",
#                               transformations=transformations,
#                               sensor_shape=(height, width),
#                               num_output_time_bins=1)

frame_input = PropheseeCamera(filename=SEQUENCE_FILENAME_RAW,
                              transformations=transformations,
                              sensor_shape=(height, width),
                              num_output_time_bins=1)

num_neurons = np.prod(frame_input.s_out.shape)
print("NUM NEURONS", num_neurons)
_, _, scaled_height, scaled_width = frame_input.s_out.shape
flat_shape = (num_neurons, )
num_out_neurons = 50

w_ff = 255
w_rec = 5
w_o = 30
kernel_size = 5

vth = 500
du = dv = 2000


print("Create FF weights")

ff_weights = csr_matrix(np.eye(num_neurons) * w_ff)

print("Create down weights")
rec_weights_down = np.zeros((num_neurons, num_neurons)).reshape((scaled_height, scaled_width, scaled_height, scaled_width))
for i in range(scaled_height):
    for j in range(scaled_width):
        rec_weights_down[i+1:i+kernel_size+1, j-kernel_size:j+kernel_size, i, j-kernel_size:j+kernel_size] = w_rec
rec_weights_down = csr_matrix(rec_weights_down.reshape((num_neurons, num_neurons)))

print("Create up weights")
rec_weights_up = np.zeros((num_neurons, num_neurons)).reshape((scaled_height, scaled_width, scaled_height, scaled_width))
for i in range(scaled_height):
    for j in range(scaled_width):
        rec_weights_up[i-kernel_size:i, j-kernel_size:j+kernel_size, i, j-kernel_size:j+kernel_size] = w_rec
rec_weights_up = csr_matrix(rec_weights_up.reshape((num_neurons, num_neurons)))

print("Create left weights")
rec_weights_left = np.zeros((num_neurons, num_neurons)).reshape((scaled_height, scaled_width, scaled_height, scaled_width))
for i in range(scaled_height):
    for j in range(scaled_width):
        rec_weights_left[i-kernel_size:i+kernel_size, j-kernel_size:j, i-kernel_size:i+kernel_size, j] = w_rec

rec_weights_left = csr_matrix(rec_weights_left.reshape((num_neurons, num_neurons)))

print("Create right weights")
rec_weights_right = np.zeros((num_neurons, num_neurons)).reshape((scaled_height, scaled_width, scaled_height, scaled_width))
for i in range(scaled_height):
    for j in range(scaled_width):
        rec_weights_right[i-kernel_size:i+kernel_size, j+1:j+kernel_size+1, i-kernel_size:i+kernel_size, j] = w_rec
rec_weights_right = csr_matrix(rec_weights_right.reshape((num_neurons, num_neurons)))

print("Create out weights")
w_out_up = np.zeros((4*num_out_neurons, num_neurons))
w_out_down = np.zeros((4*num_out_neurons, num_neurons))
w_out_left = np.zeros((4*num_out_neurons, num_neurons))
w_out_right = np.zeros((4*num_out_neurons, num_neurons))

tmp = np.kron(np.eye(num_out_neurons, dtype=np.int32), np.array([w_o] * (num_neurons // num_out_neurons)))

w_out_up[:num_out_neurons, :tmp.shape[1]] = tmp
w_out_down[num_out_neurons:2*num_out_neurons, :tmp.shape[1]] = tmp
w_out_left[num_out_neurons*2:3*num_out_neurons, :tmp.shape[1]] = tmp
w_out_right[num_out_neurons*3:, :tmp.shape[1]] = tmp

w_out_up[:, 0] = 1 # bug fix
w_out_down[:, 0] = 1 # bug fix
w_out_left[:, 0] = 1 # bug fix
w_out_right[:, 0] = 1 # bug fix

w_out_up = csr_matrix(w_out_up)
w_out_down = csr_matrix(w_out_down)
w_out_left = csr_matrix(w_out_left)
w_out_right = csr_matrix(w_out_right)



Create Cam
NUM NEURONS 768
Create FF weights
Create down weights
Create up weights
Create left weights
Create right weights
Create out weights


In [5]:

# inp_adapter = PyToNxAdapter(shape=frame_input.s_out.shape,
#                             num_message_bits=8)

print("Create inp processes")
ff_inp = Sparse(weights=ff_weights, num_message_bits=8)
lif_inp = LIF(shape=frame_input.shape,
              du=4095,
              dv=4095,
              vth=1)


print("Create down processes")
ff_down = Sparse(weights=ff_weights)
rec_down = Sparse(weights=rec_weights_down)
lif_down = LIF(shape=frame_input.shape,
              du=du,
              dv=dv,
              vth=vth)

print("Create up processes")
ff_up = Sparse(weights=ff_weights)
rec_up = Sparse(weights=rec_weights_up)
lif_up = LIF(shape=frame_input.shape,
              du=du,
              dv=dv,
              vth=vth)

print("Create left processes")
ff_left = Sparse(weights=ff_weights)
rec_left = Sparse(weights=rec_weights_left)
lif_left = LIF(shape=frame_input.shape,
              du=du,
              dv=dv,
              vth=vth)

print("Create right processes")
ff_right = Sparse(weights=ff_weights)
rec_right = Sparse(weights=rec_weights_right)
lif_right = LIF(shape=frame_input.shape,
                du=du,
                dv=dv,
                vth=vth)

print("Create out processes")

sparse_out_up = Sparse(weights=w_out_up)
sparse_out_down = Sparse(weights=w_out_down)
sparse_out_left = Sparse(weights=w_out_left)
sparse_out_right = Sparse(weights=w_out_right)

out_lif = LIF(shape=(4*num_out_neurons, ),
                 du=du,
                 dv=dv,
                 vth=vth)


out_adapter = NxToPyAdapter(shape=(4*num_out_neurons, ))



recv = VisSwipeProcess(shape=frame_input.s_out.shape,
                       direction_shape=(4*num_out_neurons, ))



Create inp processes
Create down processes
Create up processes
Create left processes
Create right processes
Create out processes


In [6]:
print("Connect")
# Connect
frame_input.s_out.connect(recv.frame_in)

# frame_input.s_out.connect(inp_adapter.inp)

frame_input.s_out.flatten().connect(ff_inp.s_in, connection_config)
ff_inp.a_out.reshape(lif_inp.a_in.shape).connect(lif_inp.a_in)

lif_inp.s_out.flatten().connect(ff_down.s_in)
ff_down.a_out.reshape(lif_down.a_in.shape).connect(lif_down.a_in)
lif_down.s_out.flatten().connect(rec_down.s_in)
rec_down.a_out.reshape(lif_down.a_in.shape).connect(lif_down.a_in)
lif_down.s_out.flatten().connect(sparse_out_down.s_in)
sparse_out_down.a_out.connect(out_lif.a_in)

lif_inp.s_out.flatten().connect(ff_up.s_in)
ff_up.a_out.reshape(lif_up.a_in.shape).connect(lif_up.a_in)
lif_up.s_out.flatten().connect(rec_up.s_in)
rec_up.a_out.reshape(lif_up.a_in.shape).connect(lif_up.a_in)
lif_up.s_out.flatten().connect(sparse_out_up.s_in)
sparse_out_up.a_out.connect(out_lif.a_in)

lif_inp.s_out.flatten().connect(ff_left.s_in)
ff_left.a_out.reshape(lif_left.a_in.shape).connect(lif_left.a_in)
lif_left.s_out.flatten().connect(rec_left.s_in)
rec_left.a_out.reshape(lif_left.a_in.shape).connect(lif_left.a_in)
lif_left.s_out.flatten().connect(sparse_out_left.s_in)
sparse_out_left.a_out.connect(out_lif.a_in)

lif_inp.s_out.flatten().connect(ff_right.s_in)
ff_right.a_out.reshape(lif_right.a_in.shape).connect(lif_right.a_in)
lif_right.s_out.flatten().connect(rec_right.s_in)
rec_right.a_out.reshape(lif_right.a_in.shape).connect(lif_right.a_in)
lif_right.s_out.flatten().connect(sparse_out_right.s_in)
sparse_out_right.a_out.connect(out_lif.a_in)


out_lif.s_out.connect(out_adapter.inp)
out_adapter.out.connect(recv.direction_in)

# out_lif.s_out.connect(recv.direction_in)



Connect



<table>
<tr>
    <td> <img src="gifs/swipe.gif" alt="Drawing" style="height: 250px;"/> </td>
</tr>
</table>

In [7]:
print("Run")
# Run
num_steps = 1000
# run_cfg = Loihi2SimCfg(exception_proc_model_map={Dense: PyDenseModelBitAcc, Sparse: PySparseModelBitAcc, LIF: PyLifModelBitAcc})
run_cfg = Loihi2HwCfg()
run_cnd = RunSteps(num_steps=num_steps)

frame_input.run(condition=run_cnd, run_cfg=run_cfg)
frame_input.stop()

Run
Per core distribution:
----------------------------------------------------------------
| AxonIn |NeuronGr| Neurons|Synapses| AxonMap| AxonMem|  Cores |
|--------------------------------------------------------------|
|    3072|       1|     200|    3224|     200|       0|       1|
|    1536|       1|     384|    7061|     384|       0|       2|
|    1536|       1|     384|    6411|     384|       0|       2|
|    1536|       1|     384|    7015|     384|       0|       2|
|    1536|       1|     384|    6526|     384|       0|       2|
|     768|       1|     768|     768|     768|       0|       1|
|--------------------------------------------------------------|
| Total                                               |      10|
----------------------------------------------------------------
VIS recv
CAM 191 10000 0.313406983


/home/pweidel/lava-nc/frameworks.ai.nx.nxsdk/nxcore/arch/n3b/n3board.py:77: UserWarning: Loihi generation overriden by environment variable LOIHI_GEN=N3C1
  warnings.warn("Loihi generation overriden by environment variable LOIHI_GEN={}".format(os.environ["LOIHI_GEN"]))


VIS 0.0 0.0 0.0 0.0
CAM send done
VIS recv
CAM 10404 8814140 9.043011435
CAM send done


Process SystemProcess-2:
Process SystemProcess-3:
Process SystemProcess-4:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/usr/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/pweidel/lava-nc/lava/src/lava/magma/runtime/message_infrastructure/multiprocessing.py", line 64, in run
    mp.Process.run(self)
  File "/home/pweidel/lava-nc/lava/src/lava/magma/runtime/message_infrastructure/multiprocessing.py", line 64, in run
    mp.Process.run(self)
  File "/usr/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/home/pweidel/lava-nc/lava/src/lava/magma/runtime/runtime.py", line 98, in target_fn
    actor.start(*args, **kwargs)
  File "/home/p

KeyboardInterrupt: 